In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:

from pyspark.sql.types import StructField, StructType, IntegerType, StringType, BooleanType, DateType, DecimalType
from pyspark.sql.functions import col, when, sum, avg, row_number,lit
from pyspark.sql.window import Window
from pyspark.sql.functions import when, col
from pyspark.sql import functions as F

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [0]:

from pyspark.sql import SparkSession 
spark = SparkSession.builder.appName("IPL Data Analysis").getOrCreate()

In [0]:
deliveries_schema = StructType([
    StructField("match_id", IntegerType(), True),
    StructField("inning", IntegerType(), True),
    StructField("batting_team", StringType(), True),
    StructField("bowling_team", StringType(), True),
    StructField("over", IntegerType(), True),
    StructField("ball", IntegerType(), True),
    StructField("batter", StringType(), True),
    StructField("bowler", StringType(), True),
    StructField("non_striker", StringType(), True),
    StructField("batsman_runs", IntegerType(), True),
    StructField("extra_runs", IntegerType(), True),
    StructField("total_runs", IntegerType(), True),
    StructField("extras_type", StringType(), True),
    StructField("is_wicket", BooleanType(), True),
    StructField("player_dismissed", StringType(), True),
    StructField("dismissal_kind", StringType(), True),
    StructField("fielder", StringType(), True)
])

In [0]:
matches_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("season", StringType(), True),
    StructField("city", StringType(), True),
    StructField("date", DateType(), True),
    StructField("match_type", StringType(), True),
    StructField("player_of_match", StringType(), True),
    StructField("venue", StringType(), True),
    StructField("team1", StringType(), True),
    StructField("team2", StringType(), True),
    StructField("toss_winner", StringType(), True),
    StructField("toss_decision", StringType(), True),
    StructField("winner", StringType(), True),
    StructField("result", StringType(), True),
    StructField("result_margin", StringType(), True),
    StructField("target_runs", StringType(), True),
    StructField("target_overs", StringType(), True),
    StructField("super_over", BooleanType(), True),
    StructField("method", StringType(), True),
    StructField("umpire1", StringType(), True),
    StructField("umpire2", StringType(), True)
])

In [0]:
matches = spark.read.schema(matches_schema).format("csv").option("header","true").load("s3://ipl-datasets-til-2024/matches.csv")


In [0]:
ball_by_ball_df=spark.read.schema(deliveries_schema).format("csv").option("header","true").load("s3://ipl-datasets-til-2024/deliveries.csv")


In [0]:
num_rows = matches.count()
num_columns=len(matches.columns)
print(f"Number of rows ={num_rows}")
print(f"Number of columns= {num_columns}")

Number of rows =1095
Number of columns= 20


In [0]:
matches.select('team1').distinct().show()


+--------------------+
|               team1|
+--------------------+
| Sunrisers Hyderabad|
|Lucknow Super Giants|
| Chennai Super Kings|
|      Gujarat Titans|
|Royal Challengers...|
|Rising Pune Super...|
|     Deccan Chargers|
|Kochi Tuskers Kerala|
|    Rajasthan Royals|
|       Gujarat Lions|
|Royal Challengers...|
|Kolkata Knight Ri...|
|Rising Pune Super...|
|     Kings XI Punjab|
|        Punjab Kings|
|       Pune Warriors|
|    Delhi Daredevils|
|      Delhi Capitals|
|      Mumbai Indians|
+--------------------+



In [0]:
# as RCB changed its name from Banglore to Bengaluru  and other similar name changes we rename 
matches = matches.withColumn(
    'team1', 
    when(col('team1') == 'Rising Pune Supergiants', 'Rising Pune Supergiant')
    .when(col('team1') == 'Royal Challengers Bangalore', 'Royal Challengers Bengaluru')
    .otherwise(col('team1'))
)

matches = matches.withColumn(
    'team2', 
    when(col('team2') == 'Rising Pune Supergiants', 'Rising Pune Supergiant')
    .when(col('team2') == 'Royal Challengers Bangalore', 'Royal Challengers Bengaluru')
    .otherwise(col('team2'))
)

matches = matches.withColumn(
    'winner', 
    when(col('winner') == 'Rising Pune Supergiants', 'Rising Pune Supergiant')
    .when(col('winner') == 'Royal Challengers Bangalore', 'Royal Challengers Bengaluru')
    .otherwise(col('winner'))
)


In [0]:
matches = matches.withColumn(
    'venue', 
    F.when(F.col('venue') == 'Feroz Shah Kotla Ground', 'Feroz Shah Kotla')
    .when(F.col('venue') == 'M Chinnaswamy Stadium', 'M. Chinnaswamy Stadium')
    .when(F.col('venue') == 'MA Chidambaram Stadium, Chepauk', 'M.A. Chidambaram Stadium')
    .when(F.col('venue') == 'M. A. Chidambaram Stadium', 'M.A. Chidambaram Stadium')
    .when(F.col('venue') == 'Punjab Cricket Association IS Bindra Stadium, Mohali', 'Punjab Cricket Association Stadium')
    .when(F.col('venue') == 'Punjab Cricket Association Stadium, Mohali', 'Punjab Cricket Association Stadium')
    .when(F.col('venue') == 'IS Bindra Stadium', 'Punjab Cricket Association Stadium')
    .when(F.col('venue') == 'Rajiv Gandhi International Stadium, Uppal', 'Rajiv Gandhi International Stadium')
    .when(F.col('venue') == 'Rajiv Gandhi Intl. Cricket Stadium', 'Rajiv Gandhi International Stadium')
    .otherwise(F.col('venue'))  # Keep the original value if no match
)

In [0]:
ball_by_ball_df.describe().show()

+-------+-----------------+------------------+-------------------+-------------------+-----------------+------------------+--------------+--------------+--------------+------------------+-------------------+------------------+-----------+----------------+--------------+--------------+
|summary|         match_id|            inning|       batting_team|       bowling_team|             over|              ball|        batter|        bowler|   non_striker|      batsman_runs|         extra_runs|        total_runs|extras_type|player_dismissed|dismissal_kind|       fielder|
+-------+-----------------+------------------+-------------------+-------------------+-----------------+------------------+--------------+--------------+--------------+------------------+-------------------+------------------+-----------+----------------+--------------+--------------+
|  count|           260920|            260920|             260920|             260920|           260920|            260920|        260920|    

In [0]:
matches = matches.na.fill(value="No team", subset=["winner"])

In [0]:
ball_by_ball_nan=ball_by_ball_df.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in ball_by_ball_df.columns])
ball_by_ball_nan.show()

+--------+------+------------+------------+----+----+------+------+-----------+------------+----------+----------+-----------+---------+----------------+--------------+-------+
|match_id|inning|batting_team|bowling_team|over|ball|batter|bowler|non_striker|batsman_runs|extra_runs|total_runs|extras_type|is_wicket|player_dismissed|dismissal_kind|fielder|
+--------+------+------------+------------+----+----+------+------+-----------+------------+----------+----------+-----------+---------+----------------+--------------+-------+
|       0|     0|           0|           0|   0|   0|     0|     0|          0|           0|         0|         0|     246795|   260920|               0|             0|      0|
+--------+------+------------+------------+----+----+------+------+-----------+------------+----------+----------+-----------+---------+----------------+--------------+-------+



In [0]:
matches=matches.replace("NA","None")

In [0]:
ball_by_ball_df.show(5)

+--------+------+--------------------+--------------------+----+----+-----------+-------+-----------+------------+----------+----------+-----------+---------+----------------+--------------+-------+
|match_id|inning|        batting_team|        bowling_team|over|ball|     batter| bowler|non_striker|batsman_runs|extra_runs|total_runs|extras_type|is_wicket|player_dismissed|dismissal_kind|fielder|
+--------+------+--------------------+--------------------+----+----+-----------+-------+-----------+------------+----------+----------+-----------+---------+----------------+--------------+-------+
|  335982|     1|Kolkata Knight Ri...|Royal Challengers...|   0|   1| SC Ganguly|P Kumar|BB McCullum|           0|         1|         1|    legbyes|     null|              NA|            NA|     NA|
|  335982|     1|Kolkata Knight Ri...|Royal Challengers...|   0|   2|BB McCullum|P Kumar| SC Ganguly|           0|         0|         0|       null|     null|              NA|            NA|     NA|
|  33

In [0]:

team_wins_data = matches.groupBy("winner").count()
team_wins_data = team_wins_data.withColumnRenamed("winner", "team") \
                               .withColumnRenamed("count", "win_count")

teams = [row["team"] for row in team_wins_data.collect()]
wins = [row["win_count"] for row in team_wins_data.collect()]


fig = go.Figure(data=[go.Bar(x=teams, y=wins)])
fig.update_layout(
    title='Teams with Most Wins in IPL',
    xaxis_title='Team',
    yaxis_title='Number of Wins',
    template='plotly_dark'
)
fig.show()


In [0]:
#Each team win percentage at each venue
venue_stats = matches.groupBy("venue", "winner").count().withColumnRenamed("count", "win_count")
venue_total_matches = matches.groupBy("venue").count().withColumnRenamed("count", "total_matches")
venue_stats = venue_stats.join(venue_total_matches, on="venue")
venue_stats = venue_stats.withColumn(
    "win_percentage", (F.col("win_count") / F.col("total_matches")) * 100
)

venue_stats.show()


+--------------------+--------------------+---------+-------------+------------------+
|               venue|              winner|win_count|total_matches|    win_percentage|
+--------------------+--------------------+---------+-------------+------------------+
|Sheikh Zayed Stadium|Kolkata Knight Ri...|        6|           29|20.689655172413794|
|    Wankhede Stadium|Rising Pune Super...|        3|           73|  4.10958904109589|
|MA Chidambaram St...|Lucknow Super Giants|        1|           28| 3.571428571428571|
|     SuperSport Park| Chennai Super Kings|        2|           12|16.666666666666664|
|Narendra Modi Sta...|      Delhi Capitals|        4|           24|16.666666666666664|
|    Wankhede Stadium|     Kings XI Punjab|        5|           73|6.8493150684931505|
|Saurashtra Cricke...| Sunrisers Hyderabad|        1|           10|              10.0|
|        Eden Gardens|      Delhi Capitals|        1|           77|1.2987012987012987|
|Dubai Internation...|        Punjab Kings|

In [0]:
import plotly.graph_objects as go
venue_stats_pd=venue_stats.toPandas()
top_venues = venue_stats_pd.groupby("venue").sum().nlargest(10, "total_matches").index
filtered_df = venue_stats_pd[venue_stats_pd["venue"].isin(top_venues)]

pivot_heatmap=filtered_df.pivot(index="venue",columns="winner",values="win_percentage").fillna(0)

fig = go.Figure(data=go.Heatmap(
    z=pivot_heatmap.values,
    x=pivot_heatmap.columns,
    y=pivot_heatmap.index, 
    colorscale="Spectral", 
    hoverongaps=False
))
fig.update_layout(
    title="Win Percentage Heatmap by Venue and Winner",
    xaxis_title="Winner",
    yaxis_title="Venue",
    width=1000,
    height=600
)
fig.show()

In [0]:
#top ten run scorer
top_10_run_scorer=ball_by_ball_df.groupby("batter").sum("batsman_runs").orderBy(sum("batsman_runs"),ascending=False)
top_10_run_scorer=top_10_run_scorer.withColumnRenamed("sum(batsman_runs)","Runs")
top_10_run_scorer=top_10_run_scorer.limit(10).toPandas()
fig = px.bar(top_10_run_scorer, x='batter', y='Runs', title='Top 10 Run Scorers', 
             labels={'batter': 'Batter', 'Runs': 'Total Runs'},color="Runs",color_continuous_scale="Viridis")
fig.show()


In [0]:
matches

Out[80]: DataFrame[id: int, season: string, city: string, date: date, match_type: string, player_of_match: string, venue: string, team1: string, team2: string, toss_winner: string, toss_decision: string, winner: string, result: string, result_margin: string, target_runs: string, target_overs: string, super_over: boolean, method: string, umpire1: string, umpire2: string]

In [0]:
matches_filtered=matches.drop("id","date","umpire1","umpire2")
matches_filtered.show(5)

+-------+----------+----------+---------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+-----------+------------+----------+------+
| season|      city|match_type|player_of_match|               venue|               team1|               team2|         toss_winner|toss_decision|              winner| result|result_margin|target_runs|target_overs|super_over|method|
+-------+----------+----------+---------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+-----------+------------+----------+------+
|2007/08| Bangalore|    League|    BB McCullum|M. Chinnaswamy St...|Royal Challengers...|Kolkata Knight Ri...|Royal Challengers...|        field|Kolkata Knight Ri...|   runs|          140|        223|          20|      null|  None|
|2007/08|Chandigarh|    League|     MEK Hussey|Punjab Cricket As...|    

In [0]:
# toss factor
toss_factor=matches_filtered.withColumn("toss_winner_is_match_winner",col('toss_winner')==col('winner'))
toss_factor_count=toss_factor.groupBy("toss_winner_is_match_winner").count().toPandas()
fig=go.Figure(data=[go.Pie(labels=toss_factor_count["toss_winner_is_match_winner"],values=toss_factor_count["count"],hole=0,textinfo='percent')])
fig.update_layout(
       
        showlegend=True,
        plot_bgcolor='black',
        paper_bgcolor='black',

        title=dict(
            text='Toss Factor',
            font=dict(size=20, color='white'),
            x=0.5,
            y=0.95
        )
    )
fig.show()



In [0]:
# toss decision and game 


In [0]:
# most out type

In [0]:
# most run in a over by bowler

In [0]:
# player specific dissmissal

In [0]:
# top 10 partnership in entire tournament

In [0]:
# top venue

In [0]:
# top wicket taker

In [0]:
# team with most extraas

In [0]:
# best bowler with economy

In [0]:
# best batsman with average

In [0]:
# batsman vs bowler insight 

In [0]:
# player performance against specific team

In [0]:
# player performance at specific ground